In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the data into a Pandas DataFrame
df_md = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_md.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [3]:
# Generate summary statistics
df_md.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_md.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

In [5]:
df_md.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, bitcoin to digibyte
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price_change_percentage_24h   41 non-null     float64
 1   price_change_percentage_7d    41 non-null     float64
 2   price_change_percentage_14d   41 non-null     float64
 3   price_change_percentage_30d   41 non-null     float64
 4   price_change_percentage_60d   41 non-null     float64
 5   price_change_percentage_200d  41 non-null     float64
 6   price_change_percentage_1y    41 non-null     float64
dtypes: float64(7)
memory usage: 2.6+ KB


---

### Prepare the Data

In [6]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming df_market_data has 'coin_id' as its index
# initialize
scaler = StandardScaler()

# fit
scaler.fit(df_md)

# predict/transform
scaled_data = scaler.transform(df_md)

# Create a new DataFrame with the scaled data
df_scaled = pd.DataFrame(scaled_data, columns=df_md.columns)

# Set the index to be the same as the original DataFrame's index (coin_id)
df_scaled.index = df_md.index

# Display the first five rows of the scaled DataFrame
df_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [7]:
# Create a a list to store inertia values and the values of k
inertia = []
k = list(range(2, 20))

# Append the value of the computed inertia from the `inertia_` attribute of teh KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_scaled)
    inertia.append(k_model.inertia_)

# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow1 = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow1.head()

,k,inertia
0,2,212.123342
1,3,165.136752
2,4,79.022435
3,5,66.413051
4,6,61.634088


In [8]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow1.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for 'k' would be 4 because it is where the variance ratio criterion peaks, which is also seen in the elbow curve plot, as it has the most prominent point on the plot. It also has a particularly good silhouette score.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [9]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [10]:
# Fit the K-Means model using the scaled data
model.fit(df_scaled)

KMeans(n_clusters=4, random_state=0)

In [11]:
# Predict the clusters to group the cryptocurrencies using the scaled data
prediction = model.predict(df_scaled)

# Print the resulting array of cluster values.
print(prediction)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [12]:
# Create a copy of the DataFrame
prediction_df = df_scaled.copy()

In [13]:
# Add a new column to the DataFrame with the predicted clusters
prediction_df["predictions"] = prediction

# Display sample data
prediction_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predictions
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [14]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.


# Create the scatter plot
scatter_plot = prediction_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="predictions",  # Color points by cluster labels
    hover_cols=["coin_id"],  # Show coin_id on hover
    title="Cryptocurrency Price Change Scatter Plot",
    xlabel="Price Change Percentage (24h)",
    ylabel="Price Change Percentage (7d)"
)

# Display the plot
scatter_plot

:NdOverlay   [predictions]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### Optimize Clusters with Principal Component Analysis.

In [15]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [16]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_result = pca.fit_transform(df_scaled)

# Create a new DataFrame with the PCA results
df_pca = pd.DataFrame(pca_result, columns=[f'PCA {i+1}' for i in range(3)], index=df_scaled.index)

# View the first five rows of the DataFrame
df_pca.head()

,PCA 1,PCA 2,PCA 3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


In [17]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_
explained_variance

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is about 90%.

In [18]:
# Create a new DataFrame with the PCA data
df_pca_named = pd.DataFrame(pca_result, columns=[f'PCA {i+1}' for i in range(3)])

# Copy the coin_id from the original DataFrame and set it as the index
df_pca_named['coin_id'] = df_pca.index  # Assuming coin_id is the index in df_features
df_pca_named.set_index('coin_id', inplace=True)

# Display sample data
df_pca_named.head()

,PCA 1,PCA 2,PCA 3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [19]:
# Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

# Append the value of the computed inertia from the `inertia_` attribute of teh KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_pca_named)
    inertia.append(k_model.inertia_)

# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow2 = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow2.head()

,k,inertia
0,1,256.874086
1,2,182.339530
2,3,135.442408
3,4,49.665497
4,5,38.672582


In [20]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow2.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for 'k' would still be 4, even when using the PCA data.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The only way in which it differs is a more pronounced point in the elbow curve at 4.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [21]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [22]:
# Fit the K-Means model using the PCA data
model.fit(df_pca)

KMeans(n_clusters=4, random_state=0)

In [23]:
# Predict the clusters to group the cryptocurrencies using the PCA data
preds = model.predict(df_pca_named)

# Print the resulting array of cluster values.
print(preds)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [24]:
# Create a copy of the DataFrame with the PCA data
pred_df=df_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
pred_df["clusters"] = preds

# Display sample data
pred_df.head()

,PCA 1,PCA 2,PCA 3,clusters
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [25]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

scatter_plot2 = pred_df.hvplot.scatter(
    x='PCA 1',
    y='PCA 2',
    by='clusters',  # Add this line to color points by K-Means labels
    hover_cols=['coin_id'],
    title='PCA Scatter Plot of Cryptocurrencies',
    xlabel='PCA 1',
    ylabel='PCA 2'
)

scatter_plot2

:NdOverlay   [clusters]
   :Scatter   [PCA 1]   (PCA 2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [26]:
# Composite plot to contrast the Elbow curves
# Create the elbow plot for the first dataset
elbow_plot_1 = df_elbow1.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve Comparison", 
    label='Elbow Plot 1',  # Label for the first elbow plot
    xticks=k
)

# Create the elbow plot for the second dataset
elbow_plot_2 = df_elbow2.hvplot.line(
    x="k", 
    y="inertia", 
    label='Elbow Plot 2',  # Label for the second elbow plot
    xticks=k
)

comp_plot1 = elbow_plot_1 * elbow_plot_2  # Use * to overlay the plots

# Display the composite plot
comp_plot1

:Overlay
   .Curve.Elbow_Plot_1 :Curve   [k]   (inertia)
   .Curve.Elbow_Plot_2 :Curve   [k]   (inertia)

In [27]:
# Composite plot to contrast the clusters
comp_plot2 = scatter_plot * scatter_plot2  # Use * to overlay the plots

# Display the composite plot
comp_plot2

:Overlay
   .NdOverlay.I  :NdOverlay   [predictions]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [clusters]
      :Scatter   [PCA 1]   (PCA 2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Reducing the number of features with k-means yields slightly improved and clearer clustering results, along with a more distinct elbow plot that clearly suggests four clusters.